In [ ]:

import geopandas as gpd
import pandas as pd

# --- 1. Carregar shapefiles ---
celulas = gpd.read_file(r"..\..\..\Notebooks\5Deployment\backend\utils\Data\Portugal_cells.gpkg")
poligonos = gpd.read_file(r"..\..\..\Data\Processed\PT-FireSprd_v2.1\L2_FireBehavior\PT-FireProg_v2.1_L2_final.shp")
total_poligonos = len(poligonos)

# ===========================
# 2. Garantir mesmo CRS
# ===========================
if celulas.crs != poligonos.crs:
    poligonos = poligonos.to_crs(celulas.crs)

# ===========================
# 3. Spatial join por interseção
# ===========================
# Aqui cada polígono pode aparecer várias vezes (uma por cada célula que intersecta)
join_inter = gpd.sjoin(poligonos, celulas, how="left", predicate="intersects")

# ===========================
# 4. Contar interseções por polígono
# ===========================
contagem = join_inter.groupby(join_inter.index).size()

# polígonos que intersectam:
poligono_1_celula = (contagem == 1).sum()
poligono_2plus_celulas = (contagem >= 2).sum()
poligono_0_celulas = (contagem.isna().sum())  # caso existam

# ===========================
# 5. Percentagens
# ===========================
pct_1 = (poligono_1_celula / total_poligonos) * 100
pct_2plus = (poligono_2plus_celulas / total_poligonos) * 100

# ===========================
# 6. Resultados
# ===========================
print("\n--- RESULTADOS ---")
print(f"Total de polígonos: {total_poligonos}")

print(f"\nPolígonos que intersectam EXATAMENTE 1 célula: {poligono_1_celula} "
      f"({pct_1:.2f}%)")

print(f"Polígonos que intersectam 2 OU MAIS células: {poligono_2plus_celulas} "
      f"({pct_2plus:.2f}%)")

if poligono_0_celulas > 0:
    print(f"\nPolígonos que não intersectam nenhuma célula: {poligono_0_celulas} "
          f"({(poligono_0_celulas/total_poligonos)*100:.2f}%)")

print("\nAnálise concluída.")

c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()



--- RESULTADOS ---
Total de polígonos: 3355

Polígonos que intersectam EXATAMENTE 1 célula: 2192 (65.34%)
Polígonos que intersectam 2 OU MAIS células: 1163 (34.66%)

Análise concluída.


In [2]:
import geopandas as gpd
import pandas as pd

# --- 1. Carregar shapefiles ---
celulas = gpd.read_file(r"..\..\Notebooks\5Deployment\backend\utils\Data\Portugal_cells.gpkg")
poligonos = gpd.read_file(r"..\..\Data\Processed\PT-FireSprd_v2.1\L2_FireBehavior\PT-FireProg_v2.1_L2_final.shp")

print("\n--- CARREGANDO DADOS ---")
print("Polígonos originais:", len(poligonos))

# ===========================
# 1.b Explodir MultiPolygons → Polygons
# ===========================
if poligonos.geom_type.isin(["MultiPolygon"]).any():
    print("\nConvertendo MultiPolygons em Polygons (explode)...")
    poligonos = poligonos.explode(ignore_index=True)
else:
    print("\nNenhum MultiPolygon encontrado.")

total_poligonos = len(poligonos)
print("Polígonos após explode:", total_poligonos)

# ===========================
# 2. Garantir mesmo CRS
# ===========================
if celulas.crs != poligonos.crs:
    print("\nCRS diferente. Reprojetando polígonos...")
    poligonos = poligonos.to_crs(celulas.crs)
else:
    print("\nCRS coincidente.")

# ===========================
# 3. Spatial join por interseção
# ===========================
print("\nCalculando interseções polígonos ↔ células...")
join_inter = gpd.sjoin(poligonos, celulas, how="left", predicate="intersects")

# ===========================
# 4. Contar interseções por polígono
# ===========================
contagem = join_inter.groupby(join_inter.index).size()

poligono_1_celula = (contagem == 1).sum()
poligono_2plus_celulas = (contagem >= 2).sum()
poligono_0_celulas = (contagem.isna().sum())

# ===========================
# 5. Percentagens
# ===========================
pct_1 = (poligono_1_celula / total_poligonos) * 100
pct_2plus = (poligono_2plus_celulas / total_poligonos) * 100
pct_0 = (poligono_0_celulas / total_poligonos) * 100 if poligono_0_celulas else 0

# ===========================
# 6. Resultados
# ===========================
print("\n--- RESULTADOS FINAIS ---")
print(f"Total de polígonos analisados: {total_poligonos}")

print(f"\nPolígonos que intersectam EXATAMENTE 1 célula: {poligono_1_celula} "
      f"({pct_1:.2f}%)")

print(f"Polígonos que intersectam 2 OU MAIS células: {poligono_2plus_celulas} "
      f"({pct_2plus:.2f}%)")

if poligono_0_celulas > 0:
    print(f"\nPolígonos que NÃO intersectam nenhuma célula: {poligono_0_celulas} "
          f"({pct_0:.2f}%)")

print("\nAnálise concluída.")



--- CARREGANDO DADOS ---
Polígonos originais: 3355

Convertendo MultiPolygons em Polygons (explode)...
Polígonos após explode: 10560

CRS diferente. Reprojetando polígonos...

Calculando interseções polígonos ↔ células...

--- RESULTADOS FINAIS ---
Total de polígonos analisados: 10560

Polígonos que intersectam EXATAMENTE 1 célula: 8825 (83.57%)
Polígonos que intersectam 2 OU MAIS células: 1735 (16.43%)

Análise concluída.
